In [15]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys
from devtools import debug

[sys.path.append(str(path)) for path in [Path.cwd(), Path.cwd().parent, Path.cwd().parent/"python"] if str(path) not in sys.path]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[]